In [1]:

import random
import math

class TableEntry:
    def __init__(self, key, value):
        self.key = key
        self.value = value
        self.in_table = True

    def get_key(self):
        return self.key

    def get_value(self):
        return self.value

    def set_value(self, value):
        self.value = value

    def is_in_table(self):
        return self.in_table

    def set_to_removed(self):
        self.key = None
        self.value = None
        self.in_table = False

    def is_removed(self):
        return (self.key is None) and (self.value is None) and not self.in_table

In [ ]:
class HashTableWithCount:
    def __init__(self, initial_capacity=512): #initializes table and capacity
        self.table_size = self.check_size(initial_capacity)
        self.table = [None] * self.table_size #table yippee
        self.probe_count = 0

    def is_prime(self, n): #primetime
        if n <= 1:
            return False
        if n <= 3:
            return True
        if n % 2 == 0 or n % 3 == 0:
            return False
        i = 5
        while i * i <= n:
            if n % i == 0 or n % (i + 2) == 0:
                return False
            i += 6
        return True

    def get_next_prime(self, n):
        while not self.is_prime(n): #finds prime that is >= to n
            n += 1
        return n

    def check_size(self, size):
        return self.get_next_prime(size) #ensures its a prime

    def get_hash_index(self, key):
        return hash(key) % self.table_size #returns initial hhash index for key

    def is_hash_table_too_full(self): #checks if table exceeds load of .7 (seems to be the right balance when I looked it up) 
        return len([entry for entry in self.table if entry is not None]) / self.table_size > 0.00001

    def enlarge_hash_table(self): #resizes table if full
        old_table = self.table
        self.table_size = self.get_next_prime(self.table_size * 2)
        self.table = [None] * self.table_size
        for entry in old_table:
            if entry and entry.is_in_table(): #rehash to new
                self.add(entry.get_key(), entry.get_value())

    def reset_probe_count(self):
        self.probe_count = 0

    def get_probes(self):
        return self.probe_count

In [ ]:
class LinearProbingWithCount(HashTableWithCount):
    def probe(self, index, key): #linear pribing to find index for key
        while self.table[index] is not None and self.table[index].get_key() != key:
            index = (index + 1) % self.table_size
            self.probe_count += 1
        return index

    def locate(self, index, key): #returns the entry at index if it matches key
        return self.table[index] if self.table[index] and self.table[index].get_key() == key else None

    def add(self, key, value): #adds key value pair to table
        if self.is_hash_table_too_full():
            self.enlarge_hash_table() #full = resize
        index = self.probe(self.get_hash_index(key), key) #find index to insert the key
        if self.table[index] is None or self.table[index].is_removed():
            self.table[index] = TableEntry(key, value) #insert new entry
        else:
            self.table[index].set_value(value)

    def remove(self, key):
        index = self.probe(self.get_hash_index(key), key)
        if self.table[index] and self.table[index].get_key() == key:
            self.table[index].set_to_removed()

    def get_value(self, key):
        index = self.probe(self.get_hash_index(key), key)
        return self.table[index].get_value() if self.table[index] else None

    def contains(self, key):
        return self.get_value(key) is not None

    def is_empty(self):
        return all(entry is None or entry.is_removed() for entry in self.table)

    def get_size(self):
        return sum(1 for entry in self.table if entry and entry.is_in_table())

    def clear(self):
        self.table = [None] * self.table_size

In [ ]:
class DoubleHashingWithCount(HashTableWithCount):
    def primary_hash(self, key):
 
        return hash(key) % self.table_size

    def secondary_hash(self, key):
        return 1 + (hash(key) % (self.table_size - 1))

    def probe(self, index, key):
        step_size = self.secondary_hash(key)
        while self.table[index] is not None and self.table[index].get_key() != key:
            index = (index + step_size) % self.table_size
            self.probe_count += 1
        return index

    def locate(self, index, key):
        return self.table[index] if self.table[index] and self.table[index].get_key() == key else None

    def add(self, key, value):
        if self.is_hash_table_too_full():
            self.enlarge_hash_table()
        index = self.probe(self.get_hash_index(key), key)
        if self.table[index] is None or self.table[index].is_removed():
            self.table[index] = TableEntry(key, value)
        else:
            self.table[index].set_value(value)

    def remove(self, key):
        index = self.probe(self.get_hash_index(key), key)
        if self.table[index] and self.table[index].get_key() == key:
            self.table[index].set_to_removed()

    def get_value(self, key):
        index = self.probe(self.get_hash_index(key), key)
        return self.table[index].get_value() if self.table[index] else None

    def contains(self, key):
        return self.get_value(key) is not None

    def is_empty(self):
        return all(entry is None or entry.is_removed() for entry in self.table)

    def get_size(self):
        return sum(1 for entry in self.table if entry and entry.is_in_table())

    def clear(self):
        self.table = [None] * self.table_size

In [23]:
class GetStatistics:
    @staticmethod
    def choose_100_names_and_add(linear_ht, double_ht, names):
        chosen_names = random.sample(names, 100)
        for name in chosen_names:
            linear_ht.add(name, "value")
            double_ht.add(name, "value")

    @staticmethod
    def get_1000_names(names):
        return random.sample(names, 1000)

    @staticmethod
    def get_10000_names(names):
        return [f"Name{i}" for i in range(10000)]

    @staticmethod
    def search_for_100_names(linear_ht, double_ht, names):
        total_linear_probes = 0
        total_double_probes = 0
        for name in names:
            linear_ht.reset_probe_count()
            double_ht.reset_probe_count()
            linear_ht.get_value(name)
            double_ht.get_value(name)
            total_linear_probes += linear_ht.get_probes()
            total_double_probes += double_ht.get_probes()
        return total_linear_probes, total_double_probes

    @staticmethod
    def main():
        table_size = 167
        linear_ht = LinearProbingWithCount(table_size)
        double_ht = DoubleHashingWithCount(table_size)

        names_1000 = GetStatistics.get_1000_names([f"Name{i}" for i in range(1000)])
        names_10000 = GetStatistics.get_10000_names([f"Name{i}" for i in range(10000)])

        linear_probe_counts = []
        double_probe_counts = []

        for _ in range(1000):
            linear_ht.clear()
            double_ht.clear()
            GetStatistics.choose_100_names_and_add(linear_ht, double_ht, names_1000)
            search_names = random.sample(names_10000, 100)
            linear_probes, double_probes = GetStatistics.search_for_100_names(linear_ht, double_ht, search_names)
            linear_probe_counts.append(linear_probes)
            double_probe_counts.append(double_probes)

        #probe count
        print("The counts for the linear table are:", linear_probe_counts)
        print("The counts for the double table are:", double_probe_counts)

        #averages
        average_linear_probes = sum(linear_probe_counts) / len(linear_probe_counts)
        average_double_probes = sum(double_probe_counts) / len(double_probe_counts)

        print(f"\nThe average number of probes for linear hashing is: {average_linear_probes:.3f}")
        print(f"The average number of probes for double hashing is: {average_double_probes:.3f}")
if __name__ == "__main__":
    GetStatistics.main()

The counts for the linear table are: [252, 178, 183, 247, 234, 326, 233, 230, 156, 265, 210, 197, 141, 213, 455, 303, 239, 203, 200, 281, 594, 171, 209, 152, 203, 229, 247, 202, 349, 377, 425, 212, 302, 248, 209, 175, 188, 200, 396, 254, 275, 244, 274, 203, 205, 210, 349, 205, 242, 322, 292, 136, 160, 262, 234, 202, 185, 306, 290, 198, 223, 191, 159, 205, 324, 213, 258, 170, 199, 240, 186, 234, 191, 161, 354, 246, 216, 187, 214, 238, 239, 282, 289, 272, 151, 213, 149, 164, 215, 306, 204, 240, 297, 287, 260, 289, 224, 285, 263, 149, 212, 220, 229, 345, 281, 227, 239, 257, 233, 220, 229, 228, 222, 239, 195, 279, 276, 360, 239, 157, 203, 174, 119, 292, 149, 261, 301, 197, 241, 243, 195, 229, 181, 434, 274, 161, 198, 371, 202, 155, 292, 189, 236, 277, 344, 256, 205, 243, 333, 248, 171, 200, 208, 191, 171, 298, 253, 372, 338, 198, 191, 184, 254, 207, 203, 285, 311, 140, 168, 205, 180, 192, 187, 210, 208, 203, 262, 422, 340, 247, 264, 289, 172, 181, 173, 152, 353, 238, 180, 284, 265, 196, 25